In [65]:
from pathlib import Path
import pandas as pd
import numpy as np
import json

def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)

In [66]:
df_gpa = pd.read_csv("https://github.com/wadefagen/datasets/raw/master/gpa/uiuc-gpa-dataset.csv").rename(columns={"Year":"year","Term":"term","Subject":"subject","Number":"number","Primary Instructor":"instructor"})

df_gpa["instructor"] = df_gpa['instructor'].str.extract(r'(\w+, \w)')

df_gpa['total_students'] = df_gpa['A+'] + df_gpa['A'] + df_gpa['A-'] + df_gpa['B'] + df_gpa['B+'] + df_gpa['B-'] + df_gpa['C+'] + df_gpa['C'] + df_gpa['C-'] + df_gpa['D+'] + df_gpa['D'] + df_gpa['D-'] + df_gpa['F']

df_gpa['gpa'] = (df_gpa['A+'] * 4 + df_gpa['A'] * 4 + df_gpa['A-'] * 3.67 + df_gpa['B'] * 3 + df_gpa['B+'] * 3.33 + df_gpa['B-'] * 2.67 + df_gpa['C+'] * 2.33 + df_gpa['C'] * 2 + df_gpa['C-'] * 1.67 + df_gpa['D+'] * 1.33 + df_gpa['D'] + df_gpa['D-'] * 0.67) / df_gpa['total_students']

df_gpa = df_gpa.groupby(["year", "term", "subject", "number", "instructor"], as_index=False).agg({"gpa": "mean", "total_students": "sum", "A+": "sum", "A": "sum", "A-": "sum", "B+": "sum", "B": "sum", "B-": "sum", "C+": "sum", "C": "sum", "C-": "sum", "D+": "sum", "D": "sum", "D-": "sum", "F": "sum"})

df_gpa["course"] = df_gpa["subject"] + " " + df_gpa["number"].astype(str)

df_gpa

,year,term,subject,number,instructor,gpa,total_students,A+,A,A-,...,B,B-,C+,C,C-,D+,D,D-,F,course
0,2010,Fall,AAS,100,"Arnaldo, C",3.463613,69,0,22,21,...,8,3,1,2,1,0,0,1,0,AAS 100
1,2010,Fall,AAS,100,"Kwon, Y",3.358982,61,6,10,14,...,5,2,3,1,0,0,0,1,1,AAS 100
2,2010,Fall,AAS,100,"Manalansan, M",3.980294,34,21,12,0,...,0,0,0,0,0,0,0,0,0,AAS 100
3,2010,Fall,AAS,100,"Winkelmann, M",3.422059,34,1,12,11,...,0,0,1,2,0,0,1,0,1,AAS 100
4,2010,Fall,AAS,120,"Lee, A",3.127315,65,8,11,3,...,13,5,2,5,1,0,4,0,0,AAS 120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39791,2020,Summer,TAM,212,"Chang, W",2.808077,52,5,7,7,...,6,6,0,5,2,3,1,0,4,TAM 212
39792,2020,Summer,TAM,251,"Kim, S",3.417083,48,5,14,8,...,10,2,1,3,1,0,0,0,0,TAM 251
39793,2020,Summer,TAM,335,"Ramlawi, N",3.097500,24,2,4,4,...,3,3,1,1,0,0,1,0,1,TAM 335
39794,2020,Summer,THEA,101,"Morrissette, J",3.586707,59,35,4,4,...,2,5,2,1,0,1,0,0,1,THEA 101


In [67]:
terms = [
    "../raw/{}-{}.csv".format(year, term)
    for year in range(df_gpa["year"].min(), df_gpa["year"].max() + 2) 
      for term in ["Winter", "Spring", "Summer", "Fall"] 
        if Path("../raw/{}-{}.csv".format(year, term)).is_file()
]

df_catalogs = pd.concat([pd.read_csv(term) for term in terms], ignore_index=True)
df_catalogs["term"] = pd.Categorical(df_catalogs["term"], ["Fall","Summer","Spring","Winter"], ordered=True)
df_catalogs.sort_values(by=["year", "term", "subject", "number", "crn", "meeting"], ascending=[False, True, True, True, True, True], ignore_index=True, inplace=True)
df_catalogs["course"] = df_catalogs["subject"] + " " + df_catalogs["number"].astype(str)

"""
df_catalogs["sectionId"] = (df_catalogs["year"].astype(str) + 
                            df_catalogs["term"].astype(str) + 
                            df_catalogs["course"].astype(str) +
                            df_catalogs["crn"].astype(str) +
                            df_catalogs["section"].astype(str))
df_catalogs["meetingId"] = (df_catalogs["year"].astype(str) + 
                            df_catalogs["term"].astype(str) + 
                            df_catalogs["crn"].astype(str) +
                            df_catalogs["section"].astype(str) +
                            df_catalogs["meeting"].astype(str))
"""

# Fix typos in descriptions
df_catalogs.loc[df_catalogs["course"]=="HIST 574", "description"]="Immerses students in major works of recent American religious history. Written from multiple disciplinary perspectives and wrestling with the knotty problems in which religion has been interwoven, these books will give the student a solid foundation in American religious history. 4 graduate hours. No professional credit."
df_catalogs.loc[df_catalogs["course"]=="ASST 104", "description"] = "Same as REL 104. See REL 104."
df_catalogs.loc[df_catalogs["course"]=="EPOL 551", "description"] = "Same as EOL 570. See EOL 570."

df_catalogs = trim_all_columns(df_catalogs).merge(df_gpa, how="left", on=["year", "term", "course", "subject", "number", "instructor"])

df_catalogs

,year,term,college,subject,subject_name,number,name,description,credit_hours,gen_ed,...,B+,B,B-,C+,C,C-,D+,D,D-,F
0,2021,Spring,KV,AAS,Asian American Studies,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,1US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021,Spring,KV,AAS,Asian American Studies,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,1SS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,Spring,KV,AAS,Asian American Studies,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,1US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,Spring,KV,AAS,Asian American Studies,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,1SS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,Spring,KV,AAS,Asian American Studies,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.,1US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415066,2010,Spring,KV,YDSH,Yiddish,420,Jewish Life-Writing,Jewish life-writing from the late 18th century...,3 OR 4 hours.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
415067,2010,Spring,KV,ZULU,Zulu,404,Intermediate Zulu II,Continuation of ZULU 403; emphasis on increasi...,4 hours.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
415068,2010,Spring,KV,ZULU,Zulu,404,Intermediate Zulu II,Continuation of ZULU 403; emphasis on increasi...,4 hours.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
415069,2010,Spring,KV,ZULU,Zulu,406,Advanced Zulu II,Continuation of Zulu 405 with increased emphas...,3 hours.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
df_colleges = (pd.read_json("../Colleges.json", orient="index")
               .reset_index()
               .rename(columns={"index": "collegeId", 0: "name"}))
df_colleges.to_csv("../neo4j/nodes/college_nodes.csv", index=False)
df_colleges

,collegeId,name
0,KL,"Agricultural, Consumer and Environmental Sciences"
1,KY,Applied Health Sciences
2,LD,Armed Forces
3,KT,College of Media
4,KW,Division of General Studies
5,KN,Education
6,KP,Engineering
7,LK,Environmental Council
8,KR,Fine and Applied Arts
9,KM,Gies College of Business


In [69]:
df_subjects = (df_catalogs[["subject", "subject_name"]]
               .drop_duplicates(ignore_index=True)
               .dropna()
               .rename(columns={"subject": "subjectId", "subject_name": "name"}))
df_subjects.to_csv("../neo4j/nodes/subject_nodes.csv", index=False)
df_subjects

,subjectId,name
0,AAS,Asian American Studies
1,ABE,Agricultural and Biological Engineering
2,ACCY,Accountancy
3,ACE,Agricultural and Consumer Economics
4,ACES,"Agricultural, Consumer and Environmental Sciences"
...,...,...
208,CINE,Cinema Studies
209,MS,Media Studies
210,ESES,"ZZZ Earth Sys, Environ & Socie"
211,LGLA,Lingala


In [70]:
df_courses = (df_catalogs[["course", "number", "name", "description", "credit_hours"]]
              .drop_duplicates(["course"], ignore_index=True)
              .dropna()
              .rename(columns={"course": "courseId", "credit_hours": "creditHours"})
              .set_index(["courseId"]))
df_bad_descriptions = df_courses.loc[df_courses["description"].str.extract(r"See\s*([A-Z]{2,4}\s*[0-9]{3})").dropna().index]
df_bad_descriptions["see_course"] = df_bad_descriptions["description"].str.extract(r"See\s*([A-Z]{2,4}\s*[0-9]{3})")[0].values
df_bad_descriptions["better_description"] = df_courses.loc[df_bad_descriptions["see_course"].values, "description"].values
df_courses.loc[df_bad_descriptions.index, "description"] = df_bad_descriptions.apply(lambda row: row["better_description"].replace(row.name, row["see_course"]), axis=1)
df_courses.reset_index(inplace=True)
df_courses.to_csv("../neo4j/nodes/course_nodes.csv", index=False)
df_courses

,courseId,number,name,description,creditHours
0,AAS 100,100,Intro Asian American Studies,Interdisciplinary introduction to the basic co...,3 hours.
1,AAS 201,201,US Racial & Ethnic Politics,Examines efforts by racial and ethnic communit...,3 hours.
2,AAS 215,215,US Citizenship Comparatively,"Examines the racial, gendered, and sexualized ...",3 hours.
3,AAS 246,246,Asian American Youth in Film,Examines both mainstream and independent films...,3 hours.
4,AAS 258,258,Muslims in America,Introduction to the study of Muslims in the Un...,3 hours.
...,...,...,...,...,...
10193,VCM 658,658,Clinical Procedure/Problem I,Course is designed to train students in physic...,1 hours.
10194,VCM 683,683,Advanced Soft Tissue Surgery,"Advanced instruction in the pathophysiology, d...",1 hours.
10195,VCM 684,684,Client Relations,"Introduction to client relations, including te...",1 hours.
10196,VCM 691,691,Adv Orthopedics Fract Fixation,Advanced instruction in the pathophysiology of...,1 hours.


In [71]:
df_sections = (df_catalogs[["crn", "year", "term", "part_of_term", 
                           "gpa", "A+", "A", "A-", "B+", "B", "B-",
                           "C+", "C", "C-", "D+", "D", "D-", "F", "sectionId",
                           "section", "section_info", "section_notes", 
                           "section_attributes", "section_capp_area", 
                           "section_co_request", "section_special_approval"]]
              .drop_duplicates()
              .dropna(subset=["sectionId"])
              .rename(columns={"part_of_term": "partOfTerm",
                              "section_info": "sectionInfo", "section_notes": "sectionNotes",
                              "section_attributes": "sectionAttributes", "section_capp_area": "sectionCappArea",
                              "section_co_request": "sectionCoRequest", "section_special_approval": "sectionSpecialApproval"}))
#df_sections[["A+", "A", "A-", "B+", "B", "B-","C+", "C", "C-", "D+", "D", "D-", "F"]] = df_sections[["A+", "A", "A-", "B+", "B", "B-","C+", "C", "C-", "D+", "D", "D-", "F"]].fillna(0)
df_sections.to_csv("../neo4j/nodes/section_nodes.csv", index=False)

df_sections

,crn,year,term,partOfTerm,gpa,A+,A,A-,B+,B,...,D-,F,sectionId,section,sectionInfo,sectionNotes,sectionAttributes,sectionCappArea,sectionCoRequest,sectionSpecialApproval
0,30106,2021,Spring,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2021SpringAAS 10030106AL1,AL1,NaN,NaN,"Social & Beh Sci - Soc Sci, and Cultural Studi...",NaN,NaN,NaN
2,30107,2021,Spring,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2021SpringAAS 10030107AD1,AD1,NaN,NaN,"Social & Beh Sci - Soc Sci, and Cultural Studi...",NaN,NaN,NaN
4,41729,2021,Spring,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2021SpringAAS 10041729AD2,AD2,NaN,NaN,"Social & Beh Sci - Soc Sci, and Cultural Studi...",NaN,NaN,NaN
6,43832,2021,Spring,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2021SpringAAS 10043832AD3,AD3,NaN,NaN,"Social & Beh Sci - Soc Sci, and Cultural Studi...",NaN,NaN,NaN
8,48232,2021,Spring,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2021SpringAAS 10048232AD4,AD4,NaN,NaN,"Social & Beh Sci - Soc Sci, and Cultural Studi...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415063,53863,2010,Spring,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2010SpringWRIT 30353863C,C,NaN,NaN,Advanced Composition course.,NaN,NaN,NaN
415065,51059,2010,Spring,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2010SpringYDSH 42051059U3,U3,This section for undergraduate students for 3 ...,NaN,NaN,NaN,NaN,NaN
415066,51060,2010,Spring,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2010SpringYDSH 42051060G4,G4,This section for graduate students for 4 credi...,Restricted to Graduate - Urbana-Champaign.,NaN,NaN,NaN,NaN
415067,30990,2010,Spring,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2010SpringZULU 40430990A,A,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
df_meetings = (df_catalogs[["start_date", "end_date", "start_time", 
                           "end_time", "type", "type_name", "meeting",
                           "days", "room", "building"]]
               .drop_duplicates()
               .dropna()
               .rename(columns={"start_date": "startDate", "end_date": "endDate", 
                                "start_time": "startTime", "end_time": "endTime",
                                "type": "typeId", "type_name": "name"}))

df_meetings.to_csv("../neo4j/nodes/meeting_nodes.csv", index=False)

df_meetings

,startDate,endDate,startTime,endTime,typeId,name,meeting,days,room,building
71,2021-01-25Z,2021-05-05Z,10:00 AM,11:20 AM,LEC,Lecture,0,MW,208,Agricultural Engr Sciences Bld
102,2021-01-25Z,2021-05-05Z,12:30 PM,01:50 PM,LCD,Lecture-Discussion,0,TR,132,Bevier Hall
103,2021-01-25Z,2021-05-05Z,02:00 PM,02:50 PM,OD,Online Discussion,1,R,208,Agricultural Engr Sciences Bld
111,2021-01-25Z,2021-05-05Z,10:00 AM,11:50 AM,LAB,Laboratory,0,R,208,Agricultural Engr Sciences Bld
112,2021-01-25Z,2021-05-05Z,10:00 AM,11:50 AM,LCD,Lecture-Discussion,1,T,208,Agricultural Engr Sciences Bld
...,...,...,...,...,...,...,...,...,...,...
415034,2010-03-15Z,2010-05-05Z,10:00 AM,11:50 AM,LAB,Laboratory,0,TR,2251,Vet Med Basic Sciences Bldg
415048,2010-01-19Z,2010-05-05Z,04:00 PM,05:20 PM,LEC,Lecture,0,W,2251,Vet Med Basic Sciences Bldg
415049,2010-01-19Z,2010-05-05Z,01:00 PM,02:50 PM,LCD,Lecture-Discussion,0,F,2271C,Vet Med Basic Sciences Bldg
415051,2010-01-19Z,2010-05-05Z,03:00 PM,04:20 PM,LCD,Lecture-Discussion,0,M,250,Veterinary Teaching Hospital


In [73]:
df_sections_meetings_instructors = (df_catalogs[["crn", "year", "term", "part_of_term", 
                                         "gpa", "A+", "A", "A-", "B+", "B", "B-",
                                         "C+", "C", "C-", "D+", "D", "D-", "F",
                                         "start_date", "end_date", "start_time", 
                                         "end_time", "type", "type_name", "instructor",
                                         "days", "room", "building", "meeting",
                                         "section", "section_info", "section_notes", 
                                         "section_attributes", "section_capp_area", 
                                         "section_co_request", "section_special_approval"]]
                            .drop_duplicates()
                            .dropna(subset=["crn", "section", "year", "term", "part_of_term", "type"])
                            .rename(columns={"part_of_term": "partOfTerm",
                                             "section_info": "sectionInfo", 
                                             "section_notes": "sectionNotes",
                                             "section_attributes": "sectionAttributes", 
                                             "section_capp_area": "sectionCappArea",
                                             "section_co_request": "sectionCoRequest", 
                                             "section_special_approval": "sectionSpecialApproval",
                                             "start_date": "startDate", "end_date": "endDate", 
                                             "start_time": "startTime", "end_time": "endTime",
                                             "type": "typeId", "type_name": "name"}))
df_sections_meetings_instructors.to_csv("../neo4j/sections_meetings_instructors.csv", index=False)
df_sections_meetings_instructors

,crn,year,term,partOfTerm,gpa,A+,A,A-,B+,B,...,room,building,meeting,section,sectionInfo,sectionNotes,sectionAttributes,sectionCappArea,sectionCoRequest,sectionSpecialApproval
0,30106,2021,Spring,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,AL1,NaN,NaN,"Social & Beh Sci - Soc Sci, and Cultural Studi...",NaN,NaN,NaN
2,30107,2021,Spring,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,AD1,NaN,NaN,"Social & Beh Sci - Soc Sci, and Cultural Studi...",NaN,NaN,NaN
4,41729,2021,Spring,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,AD2,NaN,NaN,"Social & Beh Sci - Soc Sci, and Cultural Studi...",NaN,NaN,NaN
6,43832,2021,Spring,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,AD3,NaN,NaN,"Social & Beh Sci - Soc Sci, and Cultural Studi...",NaN,NaN,NaN
8,48232,2021,Spring,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0,AD4,NaN,NaN,"Social & Beh Sci - Soc Sci, and Cultural Studi...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415066,51060,2010,Spring,1,NaN,NaN,NaN,NaN,NaN,NaN,...,313,Davenport Hall,0,G4,This section for graduate students for 4 credi...,Restricted to Graduate - Urbana-Champaign.,NaN,NaN,NaN,NaN
415067,30990,2010,Spring,1,NaN,NaN,NaN,NaN,NaN,NaN,...,243,Armory,0,A,NaN,NaN,NaN,NaN,NaN,NaN
415068,30990,2010,Spring,1,NaN,NaN,NaN,NaN,NaN,NaN,...,243,Armory,0,A,NaN,NaN,NaN,NaN,NaN,NaN
415069,52094,2010,Spring,1,NaN,NaN,NaN,NaN,NaN,NaN,...,57,Everitt Laboratory,0,A1,This course meets in 214 ISB (Intl. Studies ...,NaN,NaN,NaN,NaN,NaN


In [74]:
df_instructors = (df_catalogs[["instructor"]]
                  .drop_duplicates(ignore_index=True)
                  .dropna()
                  .rename(columns={"instructor": "name"}))
df_instructors.to_csv("../neo4j/nodes/instructor_nodes.csv", index=False)
df_instructors

,name
0,"Tabares, L"
1,"Boonsripaisal, S"
2,"Guruparan, A"
3,"Kang, Y"
4,"Wang, Y"
...,...
16538,"Pope, B"
16539,"Shchur, O"
16540,"Naughton, J"
16541,"Troutt, H"


In [75]:
df_gen_eds = (df_catalogs[["gen_ed", "gen_ed_name"]]
             .drop_duplicates(ignore_index=True)
             .dropna()
             .rename(columns={"gen_ed": "genEdId", "gen_ed_name": "name"}))
df_gen_eds.to_csv("../neo4j/nodes/gen_ed_nodes.csv", index=False)
df_gen_eds

,genEdId,name
0,1US,Cultural Studies - US Minority
1,1SS,Social & Beh Sci - Soc Sci
2,1HP,Humanities – Hist & Phil
4,1CLL,Advanced Composition
6,1NW,Cultural Studies - Non-West
7,1QR1,Quantitative Reasoning I
9,1LS,Nat Sci & Tech - Life Sciences
11,1WCC,Cultural Studies - Western
13,1LA,Humanities – Lit & Arts
18,1PS,Nat Sci & Tech - Phys Sciences


In [76]:
df_college_to_subjects = (df_catalogs[["college", "subject"]]
                         .drop_duplicates(ignore_index=True)
                         .dropna()
                         .rename(columns={"college": "collegeId", "subject": "subjectId"}))
df_college_to_subjects.to_csv("../neo4j/relationships/college_to_subjects.csv", index=False)
df_college_to_subjects

,collegeId,subjectId
0,KV,AAS
1,KL,ABE
2,KM,ACCY
3,KL,ACE
4,KL,ACES
...,...,...
211,KV,CINE
212,KT,MS
213,KV,ESES
214,KV,LGLA


In [77]:
df_subjects_to_courses = (df_catalogs[["subject", "course"]]
                         .drop_duplicates(ignore_index=True)
                         .dropna()
                         .rename(columns={"subject": "subjectId", "course": "courseId"}))
df_subjects_to_courses.to_csv("../neo4j/relationships/subjects_to_courses.csv", index=False)
df_subjects_to_courses

,subjectId,courseId
0,AAS,AAS 100
1,AAS,AAS 201
2,AAS,AAS 215
3,AAS,AAS 246
4,AAS,AAS 258
...,...,...
10198,VCM,VCM 658
10199,VCM,VCM 683
10200,VCM,VCM 684
10201,VCM,VCM 691


In [78]:
df_courses_to_sections = (df_catalogs[["course", "crn"]]
                         .drop_duplicates(ignore_index=True)
                         .dropna()
                         .rename(columns={"course": "courseId"}))
df_courses_to_sections.to_csv("../neo4j/relationships/courses_to_sections.csv", index=False)
df_courses_to_sections

,courseId,crn
0,AAS 100,30106
1,AAS 100,30107
2,AAS 100,41729
3,AAS 100,43832
4,AAS 100,48232
...,...,...
56734,VCM 684,37708
56735,VCM 691,37710
56736,VCM 691,37711
56737,VCM 697,53311


In [79]:
df_sections_to_meetings = (df_catalogs[["sectionId", "meetingId"]]
                          .drop_duplicates(ignore_index=True)
                          .dropna())
df_sections_to_meetings.to_csv("../neo4j/relationships/sections_to_meetings.csv", index=False)
df_sections_to_meetings

,sectionId,meetingId
0,2021SpringAAS 10030106AL1,2021Spring30106AL10
1,2021SpringAAS 10030107AD1,2021Spring30107AD10
2,2021SpringAAS 10041729AD2,2021Spring41729AD20
3,2021SpringAAS 10043832AD3,2021Spring43832AD30
4,2021SpringAAS 10048232AD4,2021Spring48232AD40
...,...,...
285452,2010SpringWRIT 30353863C,2010Spring53863C 1
285453,2010SpringYDSH 42051059U3,2010Spring51059U3 0
285454,2010SpringYDSH 42051060G4,2010Spring51060G4 0
285455,2010SpringZULU 40430990A,2010Spring30990A 0


In [80]:
df_gen_eds_to_courses = (df_catalogs[["course", "gen_ed"]]
                        .drop_duplicates(ignore_index=True)
                        .dropna()
                        .rename(columns={"course": "courseId", "gen_ed": "genEdId"}))
df_gen_eds_to_courses.to_csv("../neo4j/relationships/gen_eds_to_courses.csv", index=False)
df_gen_eds_to_courses

,courseId,genEdId
0,AAS 100,1US
1,AAS 100,1SS
2,AAS 201,1US
3,AAS 201,1SS
4,AAS 215,1US
...,...,...
10897,LA 215,1WCC
10904,LING 260,1US
10905,LING 260,1HP
10939,SHS 252,1US


In [81]:
df_instructors_to_meetings = (df_catalogs[["instructor", "meetingId"]]
                              .drop_duplicates(ignore_index=True)
                              .dropna()
                              .rename(columns={"course": "courseId", "section": "sectionId", "type": "typeId"}))
df_instructors_to_meetings

,instructor,meetingId
0,"Tabares, L",2021Spring30106AL10
1,"Boonsripaisal, S",2021Spring30107AD10
2,"Boonsripaisal, S",2021Spring41729AD20
3,"Guruparan, A",2021Spring43832AD30
4,"Guruparan, A",2021Spring48232AD40
...,...,...
382238,"Harris, R",2010Spring51060G4 0
382239,"Bokamba, E",2010Spring30990A 0
382240,"Hlongwa, T",2010Spring30990A 0
382241,"Bokamba, E",2010Spring52094A1 0
